###   1.Анализ производительности graph-500 на локальной машине


Определимся сначала с параметрами теста. В ходе предварительных запусков, оказалось, что на локальной машинке с 4 процессорами увеличение SСALE приводит к колоссальному увеличению времени работы (т.е. для 26 16 - всего лишь Toy размер, дождаться окончания хоть одной итерации не удалось). В итоге, я остановился на стандартном размере SCALE:16 EDGEFACTOR:16 и запускал mpi-версию в 4 потока:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;<b>mpi-run -np 4 ./graph500_mpi_simple 16 16</b><br>
В результате, по итогам 64 итераций:

То есть, по итогам теста, производительность составила: <b>median_TEPS: 5.7055e+06</b>, то есть <b>0.0057055 GTEPS</b>, что в 4 раза хуже последнего (192) места в рейтинге http://www.graph500.org/results_jul_2015
